# Machine Learning and breast cancer
                                                  Nguyen Thi Huong An - Toulouse School of Economics
In this framework, we will explore the breast cancer data and carry out the classification using PySpark.

Breast cancer data source: https://archive.ics.uci.edu/ml/datasets/breast+cancer

I will concentrate on three Machine Learning algorithms to train data. They are Logistic Regression, Gradient Boost and Random Forest.

A accurate measurement will be carried out for each method. 

And I get that Random Forest gives us the highest accuracy betwen these three methods.

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import functions as fn
from pyspark.sql.functions import mean, col, stddev
from pyspark.mllib.util import MLUtils
from pyspark.mllib.stat import Statistics
import pyspark.mllib.linalg
from pyspark.mllib.linalg import Matrix, Matrices
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.mllib.stat import KernelDensity
from pyspark.mllib.linalg import Matrices, Vectors
from pyspark.mllib.regression import LabeledPoint
from numpy import array
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel


import pandas as pd
import matplotlib.pyplot as plt
import pylab
import seaborn as sns
from scipy.stats import norm
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from string import ascii_letters
import scipy.stats as ss

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import PCA, VectorAssembler, StringIndexer
from pyspark.ml.feature import Normalizer

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline


from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.ensemble import  GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Loading the breast cancer dataset

In [4]:
# Read data file which is downloaded from this link: https://archive.ics.uci.edu/ml/datasets/breast+cancer
breastcancer = spark.read.format('csv').option("header","false").option("inferSchema","false").load('/FileStore/tables/breastcancer.csv')
# breastcancer = spark.read.csv('/FileStore/tables/breastcancer.csv', sep = ";", header = True, inferSchema = True)
display(breastcancer)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
no-recurrence-events,60-69,ge40,15-19,0-2,no,2,left,left_low,no
no-recurrence-events,50-59,premeno,25-29,0-2,no,2,left,left_low,no
no-recurrence-events,60-69,ge40,20-24,0-2,no,1,left,left_low,no
no-recurrence-events,40-49,premeno,50-54,0-2,no,2,left,left_low,no
no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,left_up,no


##### Columns Rename

In [6]:
# Add column names to the data set
breastcancer = breastcancer.withColumnRenamed("_c0", "label")
breastcancer = breastcancer.withColumnRenamed("_c1", "age")
breastcancer = breastcancer.withColumnRenamed("_c2", "menopause")
breastcancer = breastcancer.withColumnRenamed("_c3", "tumor_size")
breastcancer = breastcancer.withColumnRenamed("_c4", "inv_nodes")
breastcancer = breastcancer.withColumnRenamed("_c5", "node_caps")
breastcancer = breastcancer.withColumnRenamed("_c6", "deg_malig")
breastcancer = breastcancer.withColumnRenamed("_c7", "breast")
breastcancer = breastcancer.withColumnRenamed("_c8", "breast_quad")
breastcancer = breastcancer.withColumnRenamed("_c9", "irradiat") 

##### Let see structure of dataset

In [8]:
breastcancer.printSchema()

root
-- label: string (nullable = true)
-- age: string (nullable = true)
-- menopause: string (nullable = true)
-- tumor_size: string (nullable = true)
-- inv_nodes: string (nullable = true)
-- node_caps: string (nullable = true)
-- deg_malig: string (nullable = true)
-- breast: string (nullable = true)
-- breast_quad: string (nullable = true)
-- irradiat: string (nullable = true)

##### Verifying the abnormal value for categorical variables:

In [10]:
categorical = ['age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat', 'label']
for c in categorical:
  print(c, set(breastcancer.toPandas()[c]))
# another way
# for col in breastcancer.columns:
#     breastcancer.describe([col]).show()

age {'70-79', '60-69', '30-39', '50-59', '40-49', '20-29'}
menopause {'premeno', 'lt40', 'ge40'}
tumor_size {'15-19', '0-4', '40-44', '5-9', '45-49', '10-14', '50-54', '20-24', '35-39', '25-29', '30-34'}
inv_nodes {'9-11', '12-14', '0-2', '24-26', '6-8', '3-5', '15-17'}
node_caps {'?', 'yes', 'no'}
deg_malig {'1', '3', '2'}
breast {'left', 'right'}
breast_quad {'?', 'central', 'left_low', 'right_low', 'left_up', 'right_up'}
irradiat {'yes', 'no'}
label {'recurrence-events', 'no-recurrence-events'}

In [11]:
# delete * values as an example
# data = sqlContext.createDataFrame([("Alberto", 2, '&'), ("?", 2,'?')], ["Name", "askdaosdka", "symbole"])
# data.drop("?").show()
# df_filtered=data.filter(data.Name != '?')
# df_filtered.show(5)

+-------+----------+-------+
 Name|askdaosdka|symbole|
+-------+----------+-------+
Alberto| 2| &|
 ?| 2| ?|
+-------+----------+-------+

+-------+----------+-------+
 Name|askdaosdka|symbole|
+-------+----------+-------+
Alberto| 2| &|
+-------+----------+-------+

In [12]:
# delete ? values in our dataset
brcancer = breastcancer.filter(breastcancer.node_caps != '?')
brcancer = brcancer.filter(breastcancer.breast_quad != '?')

In [13]:
brcancer.count() # 277 observations
len(brcancer.columns) # 10 features

Out[90]: 10

In [14]:
# remove all values of ?, but it does not work perfectly. To be continued
# listOfRelevantAbnormal = breastcancer.columns
# breastcancern = ' and '.join('(%s != "?")' % col_name for col_name in listOfRelevantAbnormal)
# breastcancer.filter(breastcancern)
# type(breastcancern)


Out[11]: str

In [15]:
#verify if all abnormal values are removed
categorical = ['age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
for c in categorical:
  print(c, set(brcancer.toPandas()[c]))

age {'70-79', '60-69', '30-39', '50-59', '40-49', '20-29'}
menopause {'premeno', 'lt40', 'ge40'}
tumor_size {'15-19', '0-4', '40-44', '5-9', '45-49', '10-14', '50-54', '20-24', '35-39', '25-29', '30-34'}
inv_nodes {'9-11', '12-14', '0-2', '24-26', '6-8', '3-5', '15-17'}
node_caps {'yes', 'no'}
deg_malig {'1', '3', '2'}
breast {'left', 'right'}
breast_quad {'central', 'left_low', 'right_low', 'left_up', 'right_up'}
irradiat {'yes', 'no'}

##### I would like to see the percentage of patients with recurrence and without recurrence

In [17]:
# CLASS
import pyspark.sql.functions as fn
from pyspark.sql.window import Window
nb_class = brcancer.groupBy('label').count()
nb_class = nb_class.withColumn("percent_recurrence", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_class.show()
# There are 70% of patients without recrurrence, and 30% of patients with recurrence.


+--------------------+-----+------------------+
 label|count|percent_recurrence|
+--------------------+-----+------------------+
no-recurrence-events| 196|0.7075812274368231|
 recurrence-events| 81|0.2924187725631769|
+--------------------+-----+------------------+

##### Structure of variables

In [19]:
# AGE
nb_age = brcancer.groupBy('age').count()
nb_age = nb_age.withColumn("percent_age", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_age.show()
# The most of women got the breast cancer at [50-59], then [40-49]. It is rarely for the age of [20-29]. 
# However, the percentage of breast cancer increase considerably from 30 years old.

+-----+-----+--------------------+
 age|count| percent_age|
+-----+-----+--------------------+
30-39| 36| 0.1299638989169675|
20-29| 1|0.003610108303249...|
60-69| 55| 0.19855595667870035|
40-49| 89| 0.3212996389891697|
70-79| 5|0.018050541516245487|
50-59| 91| 0.3285198555956679|
+-----+-----+--------------------+

In [20]:
# TUMOR_SIZE
nb_tumor_size = brcancer.groupBy('tumor_size').count()
nb_tumor_size = nb_tumor_size.withColumn("percent_tumor_size", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_tumor_size.show()
# The most popular size of tumor is 30-34(21%), then 25-29(18,9%), then 20-24(17,5%). 
# The tumor size of 5-9 (small), 45-49 (big) are rare

+----------+-----+--------------------+
tumor_size|count| percent_tumor_size|
+----------+-----+--------------------+
 5-9| 4| 0.01444043321299639|
 20-24| 48| 0.17328519855595667|
 30-34| 57| 0.20577617328519857|
 15-19| 29| 0.10469314079422383|
 10-14| 28| 0.10108303249097472|
 50-54| 8| 0.02888086642599278|
 35-39| 19| 0.06859205776173286|
 45-49| 3|0.010830324909747292|
 25-29| 51| 0.18411552346570398|
 0-4| 8| 0.02888086642599278|
 40-44| 22| 0.07942238267148015|
+----------+-----+--------------------+

In [21]:
# BREAST
nb_breast = brcancer.groupBy('breast').count()
nb_breast = nb_breast.withColumn("percent_breast", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_breast.show()
#THere are 53% percent of breast cancer on the left. It is not very different from the right one.

+------+-----+-------------------+
breast|count| percent_breast|
+------+-----+-------------------+
 left| 145| 0.5234657039711191|
 right| 132|0.47653429602888087|
+------+-----+-------------------+

In [22]:
# BREAST_QUAD
nb_breast_quad = brcancer.groupBy('breast_quad').count()
nb_breast_quad = nb_breast_quad.withColumn("percent_breast_quad", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_breast_quad.show()
# THere are 38% of breast cancer on the low left, 34% on the up left.
# The percentage at the center is lowest (7,3%)

+-----------+-----+-------------------+
breast_quad|count|percent_breast_quad|
+-----------+-----+-------------------+
 left_low| 106|0.38267148014440433|
 right_up| 33|0.11913357400722022|
 central| 21|0.07581227436823104|
 left_up| 94|0.33935018050541516|
 right_low| 23|0.08303249097472924|
+-----------+-----+-------------------+

In [23]:
# DEG_MALIG
nb_deg_malig = brcancer.groupBy('deg_malig').count()
nb_deg_malig = nb_deg_malig.withColumn("percent_deg_malig", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_deg_malig.show()

+---------+-----+-------------------+
deg_malig|count| percent_deg_malig|
+---------+-----+-------------------+
 3| 82| 0.296028880866426|
 1| 66|0.23826714801444043|
 2| 129| 0.4657039711191336|
+---------+-----+-------------------+

In [24]:
# IRRADIATE (xa tri)
nb_irradiat = brcancer.groupBy('irradiat').count()
nb_irradiat = nb_irradiat.withColumn("percent_irradiat", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_irradiat.show()
# THere is 23,7% of patients who have to irradiate and 76,3% who have not to irradiate. 

+--------+-----+-------------------+
irradiat|count| percent_irradiat|
+--------+-----+-------------------+
 no| 215| 0.776173285198556|
 yes| 62|0.22382671480144403|
+--------+-----+-------------------+

In [25]:
# node_caps ()
nb_node_caps = brcancer.groupBy('node_caps').count()
nb_node_caps = nb_node_caps.withColumn("percent_node_caps", fn.col('count')/fn.sum('count').over(Window.partitionBy()))
nb_node_caps.show()
# THere are 8 missing values. 

+---------+-----+-------------------+
node_caps|count| percent_node_caps|
+---------+-----+-------------------+
 no| 221| 0.7978339350180506|
 yes| 56|0.20216606498194944|
+---------+-----+-------------------+

##### Preparing Data for Machine Learning

In [27]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel

##### From the data description, we can try to work with regression or classification.
##### It is curious about the impact of (age, breast_quad, breast, tumor_size, recurrence, DEG_MALIG) on the irradiation 
##### Or we can try an another model to see the impact of (age, breast_quad, breast, tumor_size, DEG_MALIG, irradiation) on the recurrence

In [29]:
res_target = (brcancer.groupBy("label").\
              agg(fn.count('label').alias('Headcounts'),
            ((fn.count('label')/int(brcancer.count()))*100).alias('Proportions')))

res_target = res_target.withColumn("Proportions", fn.round(res_target["Proportions"], 2))
res_target.registerTempTable("res_target")

display(sqlContext.sql("SELECT * FROM res_target"))

label,Headcounts,Proportions
no-recurrence-events,196,70.76
recurrence-events,81,29.24


In [30]:
labels = 'no-recurrence-events', 'recurrence-events'
sizes = [res_target.toPandas()['Headcounts'][0], 
         res_target.toPandas()['Headcounts'][1]]
colors = ['indianred', 'tomato']
explode = (0, 0.1) 
 
# Plot
fig, ax = plt.subplots()
ax = plt.pie(sizes, explode = explode, colors = colors, 
             autopct = '%1.1f%%', shadow = True, startangle = 140)
plt.axis('equal')
plt.title('Repartition of the recurrence-events and no-recurrence-events patients')
plt.legend(labels)

display(fig)

In [31]:
pandas_df = brcancer.select('*').toPandas()

categorical = ['age', 'menopause', 'tumor_size', 'inv_nodes',  
               'node_caps',  'deg_malig', 'breast', 'breast_quad',  
               'irradiat', 'label']
numerical = list(pandas_df.columns.difference(categorical)) + ['label']

numerical_df = pandas_df[numerical]
categorical_df = pandas_df[categorical]

We compute the correlations between the target variable and the categorical variables thanks to 'pd.factorize'.

In [33]:
corr_cat = pd.DataFrame(categorical_df.\
                       apply(lambda x : pd.factorize(x)[0]).\
                       corr(method='pearson', min_periods=1) )

display(corr_cat)

age,menopause,tumor_size,inv_nodes,node_caps,deg_malig,breast,breast_quad,irradiat,label
1.0,0.5133684827972832,0.07459816625980677,-0.06006744180597765,-0.01195759209792947,0.04779376140976157,-0.031502455625982174,-0.0026619116402183157,-0.1501490923274081,-0.13107888192049574
0.5133684827972832,1.0,-0.031069132625936004,-0.04518128980753873,-0.0485690657785434,-0.04002032218466369,-0.05921745619211069,0.02353103090480885,-0.09348062633338151,-0.08748015071577493
0.07459816625980677,-0.031069132625936004,1.0,-0.0871336306494342,-0.012161180908766068,0.054608184286876665,-0.0014290245236046938,-0.09161055703203193,0.04161151504712172,-0.10587579112225728
-0.06006744180597765,-0.04518128980753873,-0.0871336306494342,1.0,0.589656551135381,-0.32383762604992905,0.04099220478980055,-0.06362172344518985,0.32276297997829706,0.29443923168265557
-0.01195759209792947,-0.0485690657785434,-0.012161180908766068,0.589656551135381,1.0,-0.3301808787343766,-0.012345039500551745,-0.025544677281917366,0.33353716813852025,0.289002465467917
0.04779376140976157,-0.04002032218466369,0.054608184286876665,-0.32383762604992905,-0.3301808787343766,1.0,0.04587410990106866,0.04525076553653664,-0.23082133963385165,-0.3302528117153746
-0.031502455625982174,-0.05921745619211069,-0.0014290245236046938,0.04099220478980055,-0.012345039500551745,0.04587410990106866,1.0,0.1523519029139105,0.025230101908868693,-0.041303892773300495
-0.0026619116402183157,0.02353103090480885,-0.09161055703203193,-0.06362172344518985,-0.025544677281917366,0.04525076553653664,0.1523519029139105,1.0,-0.03893874725522242,-0.07724681848543101
-0.1501490923274081,-0.09348062633338151,0.04161151504712172,0.32276297997829706,0.33353716813852025,-0.23082133963385165,0.025230101908868693,-0.03893874725522242,1.0,0.22601964530445906
-0.13107888192049574,-0.08748015071577493,-0.10587579112225728,0.29443923168265557,0.289002465467917,-0.3302528117153746,-0.041303892773300495,-0.07724681848543101,0.22601964530445906,1.0


If the correlation is equal or greater than 0.5 in absolute value, it means that the variables are strongly linked. This matrix is not reallu readable. We can plot the results:

In [35]:
fig, ax = plt.subplots()

cmap = sns.diverging_palette(220, 10, as_cmap=True)
mask = np.zeros_like(corr_cat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr_cat, cmap=cmap, vmax=.3, center=0, mask=mask,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Correlation matrix between the target and the categorical variables')

display(fig)

The following variables are positively correlated with the target:
  - inv_nodes
  - node_caps
  - irradiat
  
  
The following variable are negatively correlated with the target:
  - menopause
  - age
  - deg_malig
  - breast_quad
  - tumor_size

## I. ** Logistic Regression Model **

### ** 1) Encoding of Categorical variable **

In our dataframe, we have only categorical features in it. In oder to input the features in our machine learning model, we have to transform all categorical attribute to the numeric ones by indexing them. Either it is our input features or our label column for the model, we have to do it to train our model.

For the input features of our model, name the categorical features and transformed them:

In [39]:
# https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/diabetes.csv# Transform categorical variables: Method 1
# convert panda dataframe to spark dataframe: https://dataplatform.cloud.ibm.com/exchange/public/entry/view/5ad1c820f57809ddec9a040e37b2bd55
spark_df = sqlContext.createDataFrame(categorical_df)
# transform for 'label'
indexer = StringIndexer(inputCol="label", outputCol="labelIndex")
indexed = indexer.fit(brcancer).transform(brcancer)
indexed.show(5)
indexed.select('labelIndex').distinct().show()

+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+
 label| age|menopause|tumor_size|inv_nodes|node_caps|deg_malig|breast|breast_quad|irradiat|labelIndex|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+
no-recurrence-events|30-39| premeno| 30-34| 0-2| no| 3| left| left_low| no| 0.0|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| right| right_up| no| 0.0|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| left| left_low| no| 0.0|
no-recurrence-events|60-69| ge40| 15-19| 0-2| no| 2| right| left_up| no| 0.0|
no-recurrence-events|40-49| premeno| 0-4| 0-2| no| 2| right| right_low| no| 0.0|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+
only showing top 5 rows

+----------+
labelIndex|
+----------+
 0.0|
 1.0|
+----------+

In [40]:
# transform for categorical variables
cat_col_list=['age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps',  'deg_malig', 'breast', 'breast_quad', 'irradiat']
for col in cat_col_list:
    indexer = StringIndexer(inputCol=col, outputCol=col + "Index")
    indexed = indexer.fit(indexed).transform(indexed)
indexed.show(5)

+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+
 label| age|menopause|tumor_size|inv_nodes|node_caps|deg_malig|breast|breast_quad|irradiat|labelIndex|ageIndex|menopauseIndex|tumor_sizeIndex|inv_nodesIndex|node_capsIndex|deg_maligIndex|breastIndex|breast_quadIndex|irradiatIndex|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+
no-recurrence-events|30-39| premeno| 30-34| 0-2| no| 3| left| left_low| no| 0.0| 3.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| right| right_up| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 1.0| 2.0| 0.0|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| left| left_low| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
no-recurrence-events|60-69| ge40| 15-19| 0-2| no| 2| right| left_up| no| 0.0| 2.0| 1.0| 3.0| 0.0| 0.0| 0.0| 1.0| 1.0| 0.0|
no-recurrence-events|40-49| premeno| 0-4| 0-2| no| 2| right| right_low| no| 0.0| 1.0| 0.0| 8.0| 0.0| 0.0| 0.0| 1.0| 3.0| 0.0|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+
only showing top 5 rows

In the above lines of code, we just name those features that are categorical and transformed them into numeric variables. Remember, that we didn’t overwrite the features, instead, we created new attributes by concatenating the name of previous features and the string “Index”. So that we can input only those features that we need for the training of model and keep the real one intact.

### ** 2) Typecasting of Features **

In PySpark dataframe, we have to mention the data types of the continuous feature attribute. For all the numeric variable that are not discrete, we have to typecast them to later input them in a machine learning model. However, in this model, we do not have any numeric variable.

### ** 3) Assembling of Input Features **

In this step, we actually assemble all the features we need to input in a model. We have to provide the list of those transformed categorical attributes and make a vectored feature.

In [44]:
# AsssemblerVector or OneHotEncoder ?
# An example to understand how to assemble all features
# df = spark.createDataFrame([("a","France"), ("b", "US"), ("c", "UK")], ["age","Country"])
# indexers = [StringIndexer(inputCol = col, outputCol = col+"Index") for col in df.columns]
# pipeline = Pipeline(stages = indexers)
# in_df = pipeline.fit(df).transform(df)
# in_df.show()
# feature_listT = ['age', 'Country']
# vectorAssemblerT = VectorAssembler(inputCols=feature_listT, outputCol="features")                                 
# features_vectorizedT = vectorAssemblerT.transform(in_df)
# features_vectorizedT.show()

In [45]:
# try to delete columns of categorical
# categorical_no_label = ['age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps',  'deg_malig', 'breast', 'breast_quad', 'irradiat']
# ind_df_del = ind_df
# df = ind_df_del.drop(*categorical) # df contains on ly variableIndex
# df.show(5)

+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+----------+
ageIndex|menopauseIndex|tumor_sizeIndex|inv_nodesIndex|node_capsIndex|deg_maligIndex|breastIndex|breast_quadIndex|irradiatIndex|labelIndex|
+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+----------+
 3.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0|
 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 1.0| 2.0| 0.0| 0.0|
 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2.0| 1.0| 3.0| 0.0| 0.0| 0.0| 1.0| 1.0| 0.0| 0.0|
 1.0| 0.0| 8.0| 0.0| 0.0| 0.0| 1.0| 3.0| 0.0| 0.0|
+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+----------+
only showing top 5 rows

In [46]:
# from pyspark.ml.feature import VectorAssembler
features_list = ['ageIndex','menopauseIndex', 'tumor_sizeIndex','inv_nodesIndex','node_capsIndex', 
                'deg_maligIndex', 'breastIndex', 'breast_quadIndex', 'irradiatIndex']
vectorAssembler = VectorAssembler(inputCols=features_list,
                                  outputCol="features")

features_vectorized = vectorAssembler.transform(indexed)
features_vectorized.show(5)
# see feature columns
# features_vectorized.select("features").show()

+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+
 label| age|menopause|tumor_size|inv_nodes|node_caps|deg_malig|breast|breast_quad|irradiat|labelIndex|ageIndex|menopauseIndex|tumor_sizeIndex|inv_nodesIndex|node_capsIndex|deg_maligIndex|breastIndex|breast_quadIndex|irradiatIndex| features|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+
no-recurrence-events|30-39| premeno| 30-34| 0-2| no| 3| left| left_low| no| 0.0| 3.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| (9,[0,5],[3.0,1.0])|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| right| right_up| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 1.0| 2.0| 0.0|(9,[0,2,6,7],[1.0...|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| left| left_low| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| (9,[0,2],[1.0,2.0])|
no-recurrence-events|60-69| ge40| 15-19| 0-2| no| 2| right| left_up| no| 0.0| 2.0| 1.0| 3.0| 0.0| 0.0| 0.0| 1.0| 1.0| 0.0|[2.0,1.0,3.0,0.0,...|
no-recurrence-events|40-49| premeno| 0-4| 0-2| no| 2| right| right_low| no| 0.0| 1.0| 0.0| 8.0| 0.0| 0.0| 0.0| 1.0| 3.0| 0.0|(9,[0,2,6,7],[1.0...|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+
only showing top 5 rows

### ** 4) Normalization of Input Features **

As we can observe that all of our input features are not on the same scale, so the recommended approach is to first normalize our input features then fed them into the model for the better result.

In [48]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
l1NormData = normalizer.transform(features_vectorized)
l1NormData.show(5)

+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+--------------------+
 label| age|menopause|tumor_size|inv_nodes|node_caps|deg_malig|breast|breast_quad|irradiat|labelIndex|ageIndex|menopauseIndex|tumor_sizeIndex|inv_nodesIndex|node_capsIndex|deg_maligIndex|breastIndex|breast_quadIndex|irradiatIndex| features| features_norm|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+--------------------+
no-recurrence-events|30-39| premeno| 30-34| 0-2| no| 3| left| left_low| no| 0.0| 3.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| (9,[0,5],[3.0,1.0])|(9,[0,5],[0.75,0....|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| right| right_up| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 1.0| 2.0| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.1...|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| left| left_low| no| 0.0| 1.0| 0.0| 2.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| (9,[0,2],[1.0,2.0])|(9,[0,2],[0.33333...|
no-recurrence-events|60-69| ge40| 15-19| 0-2| no| 2| right| left_up| no| 0.0| 2.0| 1.0| 3.0| 0.0| 0.0| 0.0| 1.0| 1.0| 0.0|[2.0,1.0,3.0,0.0,...|[0.25,0.125,0.375...|
no-recurrence-events|40-49| premeno| 0-4| 0-2| no| 2| right| right_low| no| 0.0| 1.0| 0.0| 8.0| 0.0| 0.0| 0.0| 1.0| 3.0| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.0...|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------+--------------+---------------+--------------+--------------+--------------+-----------+----------------+-------------+--------------------+--------------------+
only showing top 5 rows

In [49]:
# drop column with index except Label
df_combine = l1NormData
for col in features_list:
      df_combine = df_combine.drop(col)
        
df_combine.show(5)

+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------------------+--------------------+
 label| age|menopause|tumor_size|inv_nodes|node_caps|deg_malig|breast|breast_quad|irradiat|labelIndex| features| features_norm|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------------------+--------------------+
no-recurrence-events|30-39| premeno| 30-34| 0-2| no| 3| left| left_low| no| 0.0| (9,[0,5],[3.0,1.0])|(9,[0,5],[0.75,0....|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| right| right_up| no| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.1...|
no-recurrence-events|40-49| premeno| 20-24| 0-2| no| 2| left| left_low| no| 0.0| (9,[0,2],[1.0,2.0])|(9,[0,2],[0.33333...|
no-recurrence-events|60-69| ge40| 15-19| 0-2| no| 2| right| left_up| no| 0.0|[2.0,1.0,3.0,0.0,...|[0.25,0.125,0.375...|
no-recurrence-events|40-49| premeno| 0-4| 0-2| no| 2| right| right_low| no| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.0...|
+--------------------+-----+---------+----------+---------+---------+---------+------+-----------+--------+----------+--------------------+--------------------+
only showing top 5 rows

In [50]:
# drop categorical variables
for col in cat_col_list:
      df_combine = df_combine.drop(col)
        
df_combine.show(5)

+--------------------+----------+--------------------+--------------------+
 label|labelIndex| features| features_norm|
+--------------------+----------+--------------------+--------------------+
no-recurrence-events| 0.0| (9,[0,5],[3.0,1.0])|(9,[0,5],[0.75,0....|
no-recurrence-events| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.1...|
no-recurrence-events| 0.0| (9,[0,2],[1.0,2.0])|(9,[0,2],[0.33333...|
no-recurrence-events| 0.0|[2.0,1.0,3.0,0.0,...|[0.25,0.125,0.375...|
no-recurrence-events| 0.0|(9,[0,2,6,7],[1.0...|(9,[0,2,6,7],[0.0...|
+--------------------+----------+--------------------+--------------------+
only showing top 5 rows

In [51]:
#drop Label and features
df_combine = df_combine.drop("features").drop("label")
df_combine.show(5)

+----------+--------------------+
labelIndex| features_norm|
+----------+--------------------+
 0.0|(9,[0,5],[0.75,0....|
 0.0|(9,[0,2,6,7],[0.1...|
 0.0|(9,[0,2],[0.33333...|
 0.0|[0.25,0.125,0.375...|
 0.0|(9,[0,2,6,7],[0.0...|
+----------+--------------------+
only showing top 5 rows

### ** 5) Distribution of Dataset **

As we prepared our input features PySpark dataframe, now it is the right time to define our training and testing dataset to train our model on sufficient training dataset and then use unseen or test dataset to evaluate the performance of our Logistic Regression model later.

In [53]:
#Split the dataset into training and testing dataset
splits = df_combine.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

### ** 6) Configuration of the Logistic Regression Model **

Before building the machine learning pipeline, we have to make some configuration of our machine learning model using PySpark MLlib to define the structure of Logistic Regression with some initial model parameter. It’s the important step before establishing a machine learning pipeline.

In [55]:
# from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features_norm', labelCol = 'labelIndex', maxIter=10, regParam=0.3, elasticNetParam=0.8)

## ** 7) Definition of Machine Learning pipeline **

In the step of defining machine learning pipeline, we basically roll out all the stages we have prepared for the establishment of machine learning pipeline. For this purpose, we will take the instantiated logistic regression model and put this in our configured machine learning pipeline.

In [57]:
# from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[lr])

## ** 8) Train Logistic Regression Model **

For the training of our model, we will call the ‘fit’ function of a configured pipeline and then fed the training dataset as a function argument. Remember that this training dataset is different from the testing dataset, as we have split our dataset already.

In [59]:
model = pipeline.fit(df_train)

## ** 9) Prediction via Logistic Regression Model **

Once our logistic regression model is trained on training dataset, we can predict on the same training dataset that how well our model is performed on the training dataset. For the prediction, we have to call the ‘transform’ function on the trained model and then it will give us the prediction from the model.

In [61]:
prediction = model.transform(df_train)

In [62]:
prediction.printSchema()

root
-- labelIndex: double (nullable = false)
-- features_norm: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [63]:
prediction.show(5)

+----------+--------------------+--------------------+--------------------+----------+
labelIndex| features_norm| rawPrediction| probability|prediction|
+----------+--------------------+--------------------+--------------------+----------+
 0.0|(9,[0,1],[0.66666...|[0.92576947582869...|[0.71621621621621...| 0.0|
 0.0|(9,[0,1,2],[0.2,0...|[0.92576947582869...|[0.71621621621621...| 0.0|
 0.0|(9,[0,1,2],[0.285...|[0.92576947582869...|[0.71621621621621...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[0.92576947582869...|[0.71621621621621...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[0.92576947582869...|[0.71621621621621...| 0.0|
+----------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [64]:
# Evaluation on train data
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
binEval.evaluate(prediction)

Out[114]: 0.7

### ** 10) Evaluation of Testing Data **

As we have trained our model and got the simple prediction of the training dataset. Now it’s the right absolute time to evaluate our machine learning model on testing data. The testing will basically tell us that in general whether this model is too good or too bad on unseen data.

In [66]:
#test on testing data
prediction = model.transform(df_test)

In [67]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
binEval.evaluate(prediction)

Out[118]: 0.7446808510638298

##II. ** Gradient Boosting **

The second model we will perform is a Gradient Boosting. This is an aggregation model and we will see if it is better than the logistic regression.

In [69]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="labelIndex", featuresCol="features_norm", maxIter=10)


In [70]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[gbt])

In [71]:
model = pipeline.fit(df_train)

In [72]:
prediction = model.transform(df_train)

In [73]:
prediction.printSchema()

root
-- labelIndex: double (nullable = false)
-- features_norm: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [74]:
prediction.show()

+----------+--------------------+--------------------+--------------------+----------+
labelIndex| features_norm| rawPrediction| probability|prediction|
+----------+--------------------+--------------------+--------------------+----------+
 0.0|(9,[0,1],[0.66666...|[1.13600096593198...|[0.90653155375184...| 0.0|
 0.0|(9,[0,1,2],[0.2,0...|[0.89157057252952...|[0.85608430048591...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[1.06934835030730...|[0.89460779248052...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[1.06934835030730...|[0.89460779248052...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[1.08400512433775...|[0.89733981339217...| 0.0|
 0.0|(9,[0,1,2],[0.5,0...|[1.08400512433775...|[0.89733981339217...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[1.00382902716268...|[0.88159878273754...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[1.18160680494046...|[0.91397880086063...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[1.01858148520223...|[0.88464406787107...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[1.01858148520223...|[0.88464406787107...| 0.0|
 0.0|(9,[0,1,2,6],[0.2...|[0.99205836321946...|[0.87911932384305...| 0.0|
 0.0|(9,[0,1,2,6],[0.4...|[1.02720014514720...|[0.88639148903652...| 0.0|
 0.0|(9,[0,1,2,7],[0.3...|[0.98118266613035...|[0.87678870773456...| 0.0|
 0.0|(9,[0,1,2,7],[0.3...|[1.00940876964534...|[0.88275868479778...| 0.0|
 0.0|(9,[0,1,5],[0.333...|[1.03479533421467...|[0.88791222955412...| 0.0|
 0.0|(9,[0,1,5],[0.5,0...|[1.03479533421467...|[0.88791222955412...| 0.0|
 0.0|(9,[0,1,5,7],[0.3...|[1.01858148520223...|[0.88464406787107...| 0.0|
 0.0|(9,[0,1,7],[0.5,0...|[1.05577902082276...|[0.89202149416780...| 0.0|
 0.0|(9,[0,1,8],[0.5,0...|[0.77733946262493...|[0.82558848697605...| 0.0|
 0.0|(9,[0,2],[0.125,0...|[0.75968423441817...|[0.82044546559315...| 0.0|
+----------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [75]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
    
binEval.evaluate(prediction)

Out[125]: 0.9391304347826087

In [76]:
#test on testing data
prediction = model.transform(df_test)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
    
binEval.evaluate(prediction)

Out[126]: 0.723404255319149

#### III. Random Forest

In [78]:
from pyspark.mllib.tree import RandomForest
from time import *

In [79]:
start_time = time()

#model = RandomForest.trainClassifier(df_train, numClasses=2, categoricalFeaturesInfo={}, \
#    numTrees=RF_NUM_TREES, featureSubsetStrategy="auto", impurity="gini", \
#    maxDepth=RF_MAX_DEPTH, maxBins=RF_MAX_BINS, seed=RANDOM_SEED)
# train RandomForest model
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="labelIndex", featuresCol="features_norm")


end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

Time to train model: 0.008 seconds

In [80]:
pipeline = Pipeline(stages=[rf])

In [81]:
rf_model = pipeline.fit(df_train)

In [82]:
# Make predictions on train data
predictions = rf_model.transform(df_train)

In [83]:
predictions.printSchema()

root
-- labelIndex: double (nullable = false)
-- features_norm: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [84]:
predictions.show()

+----------+--------------------+--------------------+--------------------+----------+
labelIndex| features_norm| rawPrediction| probability|prediction|
+----------+--------------------+--------------------+--------------------+----------+
 0.0|(9,[0,1],[0.66666...|[16.3878901898674...|[0.81939450949337...| 0.0|
 0.0|(9,[0,1,2],[0.2,0...|[16.8469194012378...|[0.84234597006189...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[17.2112201547663...|[0.86056100773831...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[17.2112201547663...|[0.86056100773831...| 0.0|
 0.0|(9,[0,1,2],[0.333...|[17.5343073082682...|[0.87671536541341...| 0.0|
 0.0|(9,[0,1,2],[0.5,0...|[17.1641734816988...|[0.85820867408494...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[18.1201585116218...|[0.90600792558109...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[16.2184310410978...|[0.81092155205489...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[16.8874520932011...|[0.84437260466005...| 0.0|
 0.0|(9,[0,1,2,5],[0.2...|[16.8874520932011...|[0.84437260466005...| 0.0|
 0.0|(9,[0,1,2,6],[0.2...|[16.4071527449870...|[0.82035763724935...| 0.0|
 0.0|(9,[0,1,2,6],[0.4...|[16.3742458898016...|[0.81871229449008...| 0.0|
 0.0|(9,[0,1,2,7],[0.3...|[16.9137071678828...|[0.84568535839414...| 0.0|
 0.0|(9,[0,1,2,7],[0.3...|[16.3876010654725...|[0.81938005327362...| 0.0|
 0.0|(9,[0,1,5],[0.333...|[17.4047419035172...|[0.87023709517586...| 0.0|
 0.0|(9,[0,1,5],[0.5,0...|[16.8278188265941...|[0.84139094132970...| 0.0|
 0.0|(9,[0,1,5,7],[0.3...|[15.8632877615618...|[0.79316438807809...| 0.0|
 0.0|(9,[0,1,7],[0.5,0...|[15.5879326465515...|[0.77939663232757...| 0.0|
 0.0|(9,[0,1,8],[0.5,0...|[14.0638077608665...|[0.70319038804332...| 0.0|
 0.0|(9,[0,2],[0.125,0...|[16.3900128250330...|[0.81950064125165...| 0.0|
+----------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [85]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
binEval.evaluate(prediction)

Out[136]: 0.8565217391304348

In [86]:
#test on test data
prediction = rf_model.transform(df_test)

Accuracy on testing data

In [88]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("labelIndex")
binEval.evaluate(prediction)

Out[138]: 0.7446808510638298

# Conclusion:

We ran three different models: Logistic model, Gradient Boosting and Random Forest.

We got that Random Forest gives the highest accuracy for these data.